In [32]:
# Import necessary packages
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(7)
np.random.seed(7)

import helper

import matplotlib.pyplot as plt

In [2]:
### Run this cell

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [3]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)


<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [18]:
n_input  = 28*28 #784
n_hidden = 256  # 256 hiddern unit
n_out    = 10

# def softmax(x): return torch.exp(x)/torch.sum(torch.exp(x), dim=1).view(-1, 1)
# def sigmoid(x): return 1/(1+torch.exp(-x))

In [53]:
# Network V1
class Network1(nn.Module):
    def __init__(self):
        super().__init__()
        # self.input = nn.Linear(64, 784)   --> ignore this line
        self.hidden = nn.Linear(784, 256) # --> XW1 + B1
        self.output = nn.Linear(256, 10)  #
        
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x

model = Network1()
model

Network1(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
  (softmax): Softmax()
)

In [54]:
# Network V2
class Network2(nn.Module):
    def __init__(self):
        super().__init__()
        # self.input = nn.Linear(64, 784)   --> ignore this line
        self.hidden = nn.Linear(784, 256) # --> XW1 + B1
        self.output = nn.Linear(256, 10)  #
        
    def forward(self, x):
        x = F.sigmoid(self.hidden(x))
        x = F.softmax(self.output(x), dim=1)
        F.tanh
        F.re
        
        return x

model = Network2()
model

Network2(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [83]:
# Network V3
class Network3(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(784, 128)
        self.hidden2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 10)
        
    def forward(self, x):
        #x = x.view(x.shape[0], -1) # --> inputs data must be flatten
        
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.softmax(self.output(x), dim=1)
        
        return x

model = Network3()
model

Network3(
  (hidden1): Linear(in_features=784, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=10, bias=True)
)

In [56]:
print(model.hidden1.bias.shape)
print(model.hidden1.weight.shape)

torch.Size([128])
torch.Size([128, 784])


In [96]:
inputs = images.view(images.shape[0], -1) # 64x784

model = nn.Sequential(nn.Linear(784, 128),
                       nn.ReLU(),
                       nn.Linear(128, 64),
                       nn.ReLU(),
                       nn.Linear(64, 10),
                       nn.LogSoftmax(dim=1),
                     )

logit = model(inputs) # 64x784
# logit = Network3(inputs) # 64x784

# Define Loss
# criteria = nn.CrossEntropyLoss()
criteria = nn.NLLLoss()

# Calculate the loss with the logits and labels
loss = criteria(logit, labels)

In [97]:
loss

tensor(2.3041)